# Impact Analysis #

Now that we have the tool we can do some investigations:

- What is the impact of changing the data source ? 
- Does it make a difference if we take avarage rates instead of end of day rates ?

## Setup the Environmant ##
But first we need to setup the JupyterLab environment

In [1]:
%classpath config resolver maven-public http://software.pschatzmann.ch/repository/maven-public/
%classpath add mvn ch.pschatzmann:investor:0.9-SNAPSHOT
%classpath add mvn ch.pschatzmann:jupyter-jdk-extensions:0.0.1-SNAPSHOT


Added new repo: maven-public


In [2]:
// our stock evaluation framwork
import ch.pschatzmann.dates._;
import ch.pschatzmann.stocks._;
import ch.pschatzmann.stocks.data.universe._;
import ch.pschatzmann.stocks.input._;
import ch.pschatzmann.stocks.accounting._;
import ch.pschatzmann.stocks.accounting.kpi._;
import ch.pschatzmann.stocks.execution._;
import ch.pschatzmann.stocks.execution.fees._;
import ch.pschatzmann.stocks.execution.price._;
import ch.pschatzmann.stocks.parameters._;
import ch.pschatzmann.stocks.strategy._;
import ch.pschatzmann.stocks.strategy.optimization._;
import ch.pschatzmann.stocks.strategy.allocation._;
import ch.pschatzmann.stocks.strategy.selection._;
import ch.pschatzmann.stocks.integration._;
import ch.pschatzmann.stocks.strategy.OptimizedStrategy.Schedule._;

/// jupyter custom displayer
import ch.pschatzmann.display.Displayers
import ch.pschatzmann.display.Table

// java
import java.util.stream.Collectors;
import java.util._;
import java.lang._;
import java.util.function.Consumer;



import ch.pschatzmann.dates._
import ch.pschatzmann.stocks._
import ch.pschatzmann.stocks.data.universe._
import ch.pschatzmann.stocks.input._
import ch.pschatzmann.stocks.accounting._
import ch.pschatzmann.stocks.accounting.kpi._
import ch.pschatzmann.stocks.execution._
import ch.pschatzmann.stocks.execution.fees._
import ch.pschatzmann.stocks.execution.price._
import ch.pschatzmann.stocks.parameters._
import ch.pschatzmann.stocks.strategy._
import ch.pschatzmann.stocks.strategy.optimization._
import ch.pschatzmann.stocks.strategy.allocation._
import ch.pschatzmann.stocks.strategy.selection._
import ch.pschatzmann.stocks.integration._
import ch.pschatzmann.stocks.strategy.OptimizedStrategy.Schedule._
import ch.pschatzmann.display.Displayers
import ch.pschatzmann.display.Table
import ja...

## Impact of Data Source ##

First we want to investigate the impact of the data source on the result of a trading strategy. We use Apple with the RSI2 Strategy and check the result for different data readers. 

### MarketArchiveHttpReader ###

In [3]:
var account = new Account("Simulation","USD", 100000.00, Context.date("2015-01-01"), new PerTradeFees(10.0));
var stockdata = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader());
var strategy = new RSI2Strategy(stockdata);
var trader = new PaperTrader(account);
var state = new Fitness(trader).getFitness(strategy, account.getDateRange());

// print one parameter
"Return: " + state.result().getValue(KPI.AbsoluteReturn);


Return: 87513.0

### Quandl WIKI ###

In [4]:
var account = new Account("Simulation","USD", 100000.00, Context.date("2015-01-01"), new PerTradeFees(10.0));
var stockdataQ = new StockData(new StockID("AAPL", "NASDAQ"), new QuandlWIKIReader());
var strategy = new RSI2Strategy(stockdataQ);
var trader = new PaperTrader(account);
var state = new Fitness(trader).getFitness(strategy, account.getDateRange());

// print one parameter
"Return: " + state.result().getValue(KPI.AbsoluteReturn);

Return: 76282.0

### Yahoo ###

In [5]:
var account = new Account("Simulation","USD", 100000.00, Context.date("2015-01-01"), new PerTradeFees(10.0));
var stockdataYahoo = new StockData(new StockID("AAPL", "NASDAQ"), new YahooReader());
var strategy = new RSI2Strategy(stockdataYahoo);
var trader = new PaperTrader(account);
var state = new Fitness(trader).getFitness(strategy, account.getDateRange());

// print one parameter
"Return: " + state.result().getValue(KPI.AbsoluteReturn);

Return: 139969.0

### AlphaVantageReader

In [6]:
var account = new Account("Simulation","USD", 100000.00, Context.date("2015-01-01"), new PerTradeFees(10.0));
var stockdataAV = new StockData(new StockID("AAPL", "NASDAQ"), new AlphaVantageReader());
var strategy = new RSI2Strategy(stockdataAV);
var trader = new PaperTrader(account);
var state = new Fitness(trader).getFitness(strategy, account.getDateRange());

// print one parameter
"Return: " + state.result().getValue(KPI.AbsoluteReturn);

Return: 139973.0

### IEXReader

In [7]:
var account = new Account("Simulation","USD", 100000.00, Context.date("2015-01-01"), new PerTradeFees(10.0));
var stockdataIEX = new StockData(new StockID("AAPL", "NASDAQ"), new IEXReader());
var strategy = new RSI2Strategy(stockdataIEX);
var trader = new PaperTrader(account);
var state = new Fitness(trader).getFitness(strategy, account.getDateRange());

// print one parameter
"Return: " + state.result().getValue(KPI.AbsoluteReturn);

Return: 139969.0

### Charts

We can compare the three data sources on a chart. In this case we can not put the StockData into the table, because they share all the same key. We wrap them into a HistoricValues object in order to be able to assign a label as key. We could also use Indicators for this purpose instead.

In [17]:

var table = Table.create(
    HistoricValues.create(stockdata,"MarketArchive"), 
    HistoricValues.create(stockdataQ,"Quandl"),
    HistoricValues.create(stockdataAV,"AlphaVantage"),
    HistoricValues.create(stockdataIEX,"IEX"),
    HistoricValues.create(stockdataYahoo,"Yahoo")
)

new SimpleTimePlot {
    data = table.seq()
    columns = Seq("MarketArchive","Quandl","AlphaVantage","IEX","Yahoo")
    showLegend = false
}

## Impact of Selection of Rate ##
### Closing Prices ###

In [9]:
var account = new Account("Simulation","USD", 100000.00, Context.date("2015-01-01"), new PerTradeFees(10.0))
var stockdata = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader())
var strategy = new RSI2Strategy(stockdata)
var trader = new PaperTrader(account)
trader.setPrice(new ClosePrice())
var state = new Fitness(trader).getFitness(strategy, account.getDateRange())

// print one parameter
"Return: " + state.result().getValue(KPI.AbsoluteReturn)

Return: 87513.0

### Open Prices ###

In [10]:
var account = new Account("Simulation","USD", 100000.00, Context.date("2015-01-01"), new PerTradeFees(10.0))
var stockdata = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader())
var strategy = new RSI2Strategy(stockdata)
var trader = new PaperTrader(account)
trader.setPrice(new OpenPrice())
var state = new Fitness(trader).getFitness(strategy, account.getDateRange())

// print one parameter
"Return: " + state.result().getValue(KPI.AbsoluteReturn)

Return: 81297.0

### Avarege (between High and Low) Prices ###

In [11]:
var account = new Account("Simulation","USD", 100000.00, Context.date("2015-01-01"), new PerTradeFees(10.0))
var stockdata = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader())
var strategy = new RSI2Strategy(stockdata)
var trader = new PaperTrader(account)
trader.setPrice(new AvaragePrice())
var state = new Fitness(trader).getFitness(strategy, account.getDateRange())

// print one parameter
"Return: " + state.result().getValue(KPI.AbsoluteReturn)

Return: 23794.0

## Impact of Delays ##
### Same Day ##
We assume that we can do the deal at the close price for the same day. This suffers from look ahead bias because during the day the close price is not know

In [12]:
var account = new Account("Simulation","USD", 100000.00, Context.date("2015-01-01"), new PerTradeFees(10.0))
var stockdata = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader())
var strategy = new RSI2Strategy(stockdata)
var trader = new PaperTrader(account)
trader.setPrice(new ClosePrice())
var state = new Fitness(trader).getFitness(strategy, account.getDateRange())

// print one parameter
"Return: " + state.result().getValue(KPI.AbsoluteReturn)

Return: 87513.0

### Open Price of Next Day ##

We get the buy/sell signar based on the closing price and perform the trade the next day at the open price

In [13]:
var account = new Account("Simulation","USD", 100000.00, Context.date("2015-01-01"), new PerTradeFees(10.0))
var stockdata = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader())
var strategy = new RSI2Strategy(stockdata)
var trader = new PaperTrader(account)
trader.setPrice(new OpenPrice())
trader.setDelay(new OneDayDelay())
var state = new Fitness(trader).getFitness(strategy, account.getDateRange())

// print one parameter
"Return: " + state.result().getValue(KPI.AbsoluteReturn)

Return: 89494.0

### Close Price of Next Day ##

We get the buy/sell signar based on the closing price and perform the trade the next day at the close price

In [14]:
var account = new Account("Simulation","USD", 100000.00, Context.date("2015-01-01"), new PerTradeFees(10.0))
var stockdata = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader())
var strategy = new RSI2Strategy(stockdata)
var trader = new PaperTrader(account)
trader.setPrice(new ClosePrice())
trader.setDelay(new OneDayDelay())
var state = new Fitness(trader).getFitness(strategy, account.getDateRange())

// print one parameter
"Return: " + state.result().getValue(KPI.AbsoluteReturn)

Return: 93475.0

### Avarage Price of Next Day ##

We get the buy/sell signar based on the closing price and perform the trade the next day at the close price

In [15]:
var account = new Account("Simulation","USD", 100000.00, Context.date("2015-01-01"), new PerTradeFees(10.0))
var stockdata = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader())
var strategy = new RSI2Strategy(stockdata)
var trader = new PaperTrader(account)
trader.setPrice(new AvaragePrice())
trader.setDelay(new OneDayDelay())
var state = new Fitness(trader).getFitness(strategy, account.getDateRange())

// print one parameter
"Return: " + state.result().getValue(KPI.AbsoluteReturn)

Return: 28101.0